In [3]:
from datasets import load_dataset
import random
import os
import cv2
import numpy as np
import requests
from tqdm import tqdm


c:\Users\meijg\anaconda3\envs\colorize\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:

# Download de dataset (112k trainingsbeelden)
ds = load_dataset("nickpai/coco2017-colorization", split="train", revision="main")
print(f"Totaal aantal beelden: {len(ds)}")  # ~112.268 :contentReference[oaicite:2]{index=2}

Totaal aantal beelden: 112268


In [40]:
N = 100
indices = random.sample(range(len(ds)), N)
subset = ds.select(indices)
print(f"Subset grootte: {len(subset)}")


Subset grootte: 100


In [41]:
# Save the subset to a new dataset
subset.save_to_disk("coco2017-colorization-subset")

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 6270.73 examples/s]


In [42]:
print(subset[0])

{'license': 2, 'file_name': 'train2017/000000041773.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000041773.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-17 03:02:14', 'flickr_url': 'http://farm5.staticflickr.com/4030/4623906560_212407aa92_z.jpg', 'image_id': 41773, 'ids': [171989, 172100, 174920, 176492, 178520], 'captions': ['A road sign on the side of a road.', 'a black and yellow sign a road a hill and some bushes', 'A yellow diamond shape sign indicating duck crossing. ', 'A roadway with green foliage on the side and a sign that says "Nene Crossing".', 'A sign warning the crossing of birds on a road. ']}


In [43]:


resize_to = (256, 256)
output_dir = "C:/Users/meijg/Colorize/data/images"
os.makedirs(output_dir, exist_ok=True)

print("⬇️ Downloaden en resizen van afbeeldingen...")
for example in tqdm(subset):
    url = example["coco_url"]
    filename = os.path.basename(url)
    save_path = os.path.join(output_dir, filename)

    if not os.path.exists(save_path):
        try:
            # Download image bytes
            response = requests.get(url, timeout=10)
            img_array = np.frombuffer(response.content, np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

            if img is None:
                print(f"⚠️ Ongeldige afbeelding: {url}")
                continue

            # Resize en opslaan
            resized_img = cv2.resize(img, resize_to)
            cv2.imwrite(save_path, resized_img, [cv2.IMWRITE_JPEG_QUALITY, 90])

        except Exception as e:
            print(f"❌ Fout bij {url}: {e}")

print(f"✅ Klaar! Afbeeldingen opgeslagen in: {output_dir}")


⬇️ Downloaden en resizen van afbeeldingen...


100%|██████████| 100/100 [01:00<00:00,  1.66it/s]

✅ Klaar! Afbeeldingen opgeslagen in: C:/Users/meijg/Colorize/data/images


In [7]:
lab_output_dir = "C:/Users/meijg/Colorize/data/lab_images"
input_dir = "C:/Users/meijg/Colorize/data/images"
os.makedirs(lab_output_dir, exist_ok=True)

print("⬇️ Converteer naar LAB en sla op als .npy...")

for filename in tqdm(os.listdir(input_dir)):
    input_path = os.path.join(input_dir, filename)
    lab_path = os.path.join(lab_output_dir, os.path.splitext(filename)[0] + ".npy")

    if not os.path.exists(lab_path):
        try:
            img = cv2.imread(input_path)
            if img is None:
                print(f"⚠️ Ongeldige afbeelding: {input_path}")
                continue

            lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            np.save(lab_path, lab_img)

        except Exception as e:
            print(f"❌ Fout bij {input_path}: {e}")

⬇️ Converteer naar LAB en sla op als .npy...


100%|██████████| 100/100 [00:00<00:00, 779.56it/s]


In [10]:
# testing to change back to RGB
test_image_path = "C:/Users/meijg/Colorize/data/lab_images/000000005453.npy" 
test_image = np.load(test_image_path)
test_image_rgb = cv2.cvtColor(test_image, cv2.COLOR_LAB2BGR)
cv2.imwrite("C:/Users/meijg/Colorize/data/test/test.jpg", test_image_rgb)
print("✅ Klaar! LAB-afbeeldingen opgeslagen in:", lab_output_dir)

✅ Klaar! LAB-afbeeldingen opgeslagen in: C:/Users/meijg/Colorize/data/lab_images


array([[[ 27,  19,  11],
        [ 27,  19,  11],
        [ 27,  19,  11],
        ...,
        [ 85,  91,  94],
        [ 91,  97,  98],
        [ 93,  99,  97]],

       [[ 27,  19,  11],
        [ 27,  19,  11],
        [ 27,  19,  11],
        ...,
        [ 85,  91,  94],
        [ 91,  97,  98],
        [ 92,  99, 100]],

       [[ 27,  19,  11],
        [ 27,  19,  11],
        [ 29,  20,  14],
        ...,
        [ 90,  93,  96],
        [ 91,  97,  98],
        [ 92,  99, 100]],

       ...,

       [[ 16,  32,  60],
        [ 16,  32,  60],
        [ 16,  32,  60],
        ...,
        [140, 140, 142],
        [138, 138, 144],
        [138, 138, 144]],

       [[ 15,  30,  57],
        [ 15,  30,  57],
        [ 15,  30,  59],
        ...,
        [140, 140, 146],
        [138, 138, 144],
        [138, 138, 140]],

       [[ 16,  30,  55],
        [ 15,  30,  57],
        [ 15,  30,  59],
        ...,
        [140, 140, 146],
        [138, 138, 144],
        [138, 138, 140]]